# AN2DL - First Challenge

## Initial Operations

### Import the libraries

In [ ]:
import os
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tfk = tf.keras
tfkl = tf.keras.layers

### Set random seed for reproducibility

In [ ]:
# Random seed for reproducibility
seed = 69

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Metadata

In [ ]:
training_dir = './GitHub/LeafClassification-AN2DL/Data/dataset_no_corrupted/training'
validation_dir = './GitHub/LeafClassification-AN2DL/Data/dataset_no_corrupted/validation'

In [ ]:
input_shape = (256, 256, 3)

## Model - Transfer Learning from InceptionResNetV2

### Data Generators

In [ ]:
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

train_data_gen = ImageDataGenerator(rotation_range=90,
                                    height_shift_range=100,
                                    width_shift_range=100,
                                    zoom_range=0.5,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    shear_range = 0.25,
                                    fill_mode='reflect',
                                    brightness_range=[0.5,1.5],
                                    preprocessing_function = preprocess_input)
val_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)

train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None,
                                               class_mode='categorical',
                                               batch_size=128,
                                               shuffle=True,
                                               seed=seed)
validation_gen = val_data_gen.flow_from_directory(directory=validation_dir,
                                           target_size=(256,256),
                                           color_mode='rgb',
                                           classes=None,
                                           class_mode='categorical',
                                           batch_size=128,
                                           shuffle=True,
                                           seed=seed)

Found 16167 images belonging to 14 classes.
Found 1540 images belonging to 14 classes.


### Class Weight (for Unbalanced Classes)

In [ ]:
classes = dict()
for label in sorted(os.listdir(training_dir)):
  classes[label] = len(os.listdir(training_dir + "/" + label))
total = 0
class_weight = dict()
for i, samples_number in enumerate(classes.values()):
    class_weight[i] = 1/samples_number
    total += samples_number
class_weight = {key:value*total/14 for key, value in class_weight.items()}

### Model Definition

#### Supernet (InceptionResNetV2)

In [ ]:
# Download the supernet
supernet = tfk.applications.inception_resnet_v2.InceptionResNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3)
)
# Fine Tuning - Freeze the first layers
supernet.trainable = True
for i, layer in enumerate(supernet.layers[:460]):
    layer.trainable=False

2021-11-26 08:22:08.443463: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-26 08:22:08.533731: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-26 08:22:08.534471: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-26 08:22:08.535698: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

219070464/219055592 [==============================] - 1s 0us/step


#### Additional Dense Layers

In [ ]:
# Build the model
input_layer = tfk.Input(shape=input_shape)
features_extractor = supernet(input_layer)
flattening = tfkl.Flatten(name='Flattening')(features_extractor)
dropuout_flattening = tfkl.Dropout(0.2, seed=seed)(flattening)
dense1 = tfkl.Dense(
    512, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(dropuout_flattening)
dropout_dense1 = tfkl.Dropout(0.3, seed=seed)(dense1)
dense2 = tfkl.Dense(
    480, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(dropout_dense1)
dropout_dense2 = tfkl.Dropout(0.2, seed=seed)(dense2)
dense3 = tfkl.Dense(
    192, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(dropout_dense2)
dropout_dense3 = tfkl.Dropout(0.15, seed=seed)(dense3)
output_layer = tfkl.Dense(
    14, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(dropout_dense3)
model = tfk.Model(inputs=input_layer, outputs=output_layer, name='InceptionResNetV2_SameLR')

### Model Training

#### Callbacks

In [ ]:
def callbacks(dir,checkpoint = False):
    callbacks = []
    if checkpoint:
        # Checkpoints
        checkpoints_dir = os.path.join(dir,"Checkpoint")
        if not os.path.exists(checkpoints_dir):
            os.makedirs(checkpoints_dir)
        checkpoint = tfk.callbacks.ModelCheckpoint(filepath=os.path.join(checkpoints_dir,"Model.hdf5"),
                                                   monitor = "accuracy",
                                                   save_weights_only=False,
                                                   save_best_only=True
                                                  )
        callbacks.append(checkpoint)
    # Tensorboard
    tensorboard_dir = os.path.join(dir,"TensorBoard")
    if not os.path.exists(tensorboard_dir):
        os.makedirs(tensorboard_dir)
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir=tensorboard_dir,
                                                 profile_batch=0,
                                                 histogram_freq=1)
    callbacks.append(tensorboard)
    
    return callbacks

#### First Run (Bigger Learning Rate)

In [ ]:
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate = 1e-4), metrics='accuracy')
history = model.fit(
    x = train_gen,
    epochs = 15,
    validation_data = validation_gen,
    callbacks = callbacks('First'),
    class_weight=class_weight
).history

2021-11-26 08:22:21.376268: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15


2021-11-26 08:22:36.972212: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


127/127 [==============================] - 393s 3s/step - loss: 1.5446 - accuracy: 0.4604 - val_loss: 0.5946 - val_accuracy: 0.8500


2021-11-26 08:29:04.140406: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3397386240 exceeds 10% of free system memory.


...

Epoch 4/15
127/127 [==============================] - 346s 3s/step - loss: 0.1620 - accuracy: 0.9416 - val_loss: 0.0199 - val_accuracy: 0.9942


2021-11-26 08:47:26.622153: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3397386240 exceeds 10% of free system memory.


Epoch 5/15
127/127 [==============================] - 346s 3s/step - loss: 0.1261 - accuracy: 0.9555 - val_loss: 0.0275 - val_accuracy: 0.9935


2021-11-26 08:53:47.416105: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3397386240 exceeds 10% of free system memory.


Epoch 6/15
127/127 [==============================] - 345s 3s/step - loss: 0.1005 - accuracy: 0.9608 - val_loss: 0.0096 - val_accuracy: 0.9961
Epoch 7/15
127/127 [==============================] - 346s 3s/step - loss: 0.0940 - accuracy: 0.9685 - val_loss: 0.0573 - val_accuracy: 0.9857
Epoch 8/15
127/127 [==============================] - 353s 3s/step - loss: 0.0928 - accuracy: 0.9641 - val_loss: 0.0093 - val_accuracy: 0.9968
Epoch 9/15
127/127 [==============================] - 356s 3s/step - loss: 0.0885 - accuracy: 0.9686 - val_loss: 0.2590 - val_accuracy: 0.9584
Epoch 10/15
127/127 [==============================] - 352s 3s/step - loss: 0.0833 - accuracy: 0.9689 - val_loss: 0.0282 - val_accuracy: 0.9948
Epoch 11/15
127/127 [==============================] - 348s 3s/step - loss: 0.0846 - accuracy: 0.9713 - val_loss: 0.0179 - val_accuracy: 0.9942
Epoch 12/15
127/127 [==============================] - 346s 3s/step - loss: 0.0712 - accuracy: 0.9703 - val_loss: 0.0222 - val_accuracy: 0.9

#### Second Run (Smaller Learning Rate and Bigger Epsilon)

In [ ]:
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate = 1e-5,epsilon = 1e-4), metrics='accuracy')
history = model.fit(
    x = train_gen,
    epochs = 20,
    validation_data = validation_gen,
    callbacks = callbacks('Second',checkpoint = True),
    class_weight=class_weight
).history

Epoch 1/20
127/127 [==============================] - 363s 3s/step - loss: 0.0535 - accuracy: 0.9816 - val_loss: 0.0058 - val_accuracy: 0.9994
Epoch 2/20
127/127 [==============================] - 349s 3s/step - loss: 0.0438 - accuracy: 0.9834 - val_loss: 0.0044 - val_accuracy: 0.9994
Epoch 3/20
127/127 [==============================] - 345s 3s/step - loss: 0.0415 - accuracy: 0.9852 - val_loss: 0.0042 - val_accuracy: 0.9994
Epoch 4/20
127/127 [==============================] - 347s 3s/step - loss: 0.0359 - accuracy: 0.9853 - val_loss: 0.0043 - val_accuracy: 0.9994
Epoch 5/20
127/127 [==============================] - 346s 3s/step - loss: 0.0313 - accuracy: 0.9876 - val_loss: 0.0042 - val_accuracy: 0.9994
Epoch 6/20
127/127 [==============================] - 345s 3s/step - loss: 0.0307 - accuracy: 0.9863 - val_loss: 0.0041 - val_accuracy: 0.9994
Epoch 7/20
127/127 [==============================] - 347s 3s/step - loss: 0.0303 - accuracy: 0.9869 - val_loss: 0.0046 - val_accuracy: 0.9994